# Aula 4

Importando os pacotes necessários:

In [ ]:
import pandas as pd
import seaborn as sns
import networkx as nx

import freeman as fm

Operacionalizando as fórmulas do [material](https://ensino.hashi.pro.br/redesoc/material/lacunas.html):

In [ ]:
from math import log

def proportion(g, n, m):
    return 1 / g.degree(n)

def local_effsize(g, n, m):
    s = 1
    for k in g.neighbors(n):
        if g.has_edge(k, m):
            s -= proportion(g, n, k)
    return s

def effsize(g, n):
    s = 0
    for m in g.neighbors(n):
        s += local_effsize(g, n, m)
    return s

def local_constraint(g, n, m):
    s = proportion(g, n, m)
    for k in g.neighbors(n):
        if g.has_edge(k, m):
            s += proportion(g, n, k) * proportion(g, k, m)
    return s**2

def constraint(g, n):
    if g.degree(n) == 0:
        return 2
    s = 0
    for m in g.neighbors(n):
        s += local_constraint(g, n, m)
    return s

def hierarchy(g, n):
    c = constraint(g, n)
    N = g.number_of_nodes()
    s = 0
    for m in g.neighbors(n):
        f = local_constraint(g, n, m) / (c / N)
        s += f * log(f)
    return s / (N * log(N))

Carregando a rede de contribuiçao intelectual:

In [ ]:
g = fm.load('intellect.gml')

Calculando métricas conhecidas:

In [ ]:
print('calculando centralidade degree...')
dc = nx.degree_centrality(g)

print('calculando centralidade betweenness...')
bc = nx.betweenness_centrality(g)

print('calculando centralidade closeness...')
cc = nx.closeness_centrality(g)

print('calculando centralidade eigenvector...')
ec = nx.eigenvector_centrality(g)

Carregando métricas novas:

In [ ]:
print('calculando efficient size...')
eff = {}
for n in g.nodes:
    eff[n] = effsize(g, n)

print('calculando constraint...')
con = {}
for n in g.nodes:
    con[n] = constraint(g, n)

print('calculando hierarchy...')
hie = {}
for n in g.nodes:
    hie[n] = hierarchy(g, n)

Armazenando em dataframe:

In [ ]:
data = pd.DataFrame({
    'nodes': [n for n in g.nodes],
    'affiliation': [g.nodes[n]['affiliation'] for n in g.nodes],
    'gender': ['male' if g.nodes[n]['gender'] == 1 else 'female' for n in g.nodes],
    'degree centrality': [dc[n] for n in g.nodes],
    'betweenness centrality': [bc[n] for n in g.nodes],
    'closeness centrality': [cc[n] for n in g.nodes],
    'eigenvector centrality': [ec[n] for n in g.nodes],
    'efficient size': [eff[n] for n in g.nodes],
    'constraint': [con[n] for n in g.nodes],
    'hierarchy': [hie[n] for n in g.nodes],
})

fm.resize_all_plots(800, 1000)

Verificando a relação entre métricas:

In [ ]:
sns.scatterplot(data=data, x='constraint', y='betweenness centrality');

In [ ]:
sns.scatterplot(data=data, x='degree centrality', y='efficient size');

Plotando métricas conhecidas:

In [ ]:
sns.boxplot(data=data, x='degree centrality', y='affiliation');

In [ ]:
sns.boxplot(data=data, x='betweenness centrality', y='affiliation');

In [ ]:
sns.boxplot(data=data, x='closeness centrality', y='affiliation');

In [ ]:
sns.boxplot(data=data, x='eigenvector centrality', y='affiliation');

Plotando métricas novas:

In [ ]:
sns.boxplot(data=data, x='efficient size', y='affiliation');

In [ ]:
sns.boxplot(data=data, x='constraint', y='affiliation');

In [ ]:
sns.boxplot(data=data, x='hierarchy', y='affiliation');

EOF